- target-folder
    - English_EN
        - MaxiCharger AC
            - Wallbox
                - Installation and Operation Manual
                    - arch
                - Quick Guide
                    - arch
                - Pedestal EP
                    - Quick Guide
                        - arch
            - Compact
                - ...
            - Ultra
                - ...
            - Maintenance Manual
                - arch
            - Pedestal
                - Quick Guide
                    - arch
        - MaxiCharger DC
            - Fast
                - Installation and Operation Manual
                    - arch
                - Service Manual
                    - arch
                - CMS
                    - Quick Gduie
                        - arch
            - Compact
                - ...
        - Energy Management System
            - Operation and Installation Manul
                - arch
    - German_DE
        - ...


### Test Folder Description
Under folder *"target-folder\English_EN\MaxiCharger AC\Ultra\User Manual"* there are 3 pdf-files and 1 txt-files,<br>
one of the pdf-files has unregular name and should not be identified,<br>
only the pdf-file that edited on 2023-07-06 should be verified.<br>
<br>
The document under folder *"target-folder\English_EN\MaxiCharger DC\Fast\CMS\Quick Guide"* does not exist,<br>
so that this document should be recognized as *missing*<br>
<br>
Under folder *"target-folder\French_FR\MaxiCharger AC\Wallbox\Installation and Operation Manual"* there is one pdf-files named as *"User Mnual"*,<br>
which does not comply with the naming regulation and should be recognized as *abnormal*<br>
<br>
In summary, 15 documents shall be available, but only 14 documents will be found, 1 of those is abnormal.

In [27]:
import os
import re

verif = False
doc_num = 0
fund_num = 0

for root, dirs, files in os.walk(os.path.join('target-folder', 'English_EN')):
    if "arch" in dirs:
        doc_num += 1
        print(root, "...", dirs, "...", files)
        pattern_1 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))(.*\\)(.+ (Guide|Manual))$"
        pattern_2 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(Energy Management System)(.*\\)(.+ (Guide|Manual))$"
        match_1 = re.match(pattern_1, root)
        match_2 = re.match(pattern_2, root)
        if match_1:
            category = match_1.group(3)
            item = match_1.group(5).replace("\\", "").strip()
            document = match_1.group(6).replace("\\", "") 
        elif match_2:
            category = match_2.group(3)
            item = ''
            document = match_2.group(5).replace("\\", "") 
        else:
            print("Document path '%s' not correct." % (root))
        if len(files)!=0:
            fund_num += 1
            #print("Document {0} {1} {2} exists.".format(category, item, document))
        else:
            print("Document {0} {1} {2} is missing.".format(category, item, document))
    else:
        pass

print("%d documents shall be available." % (doc_num))
print("%d documents has been found." % (fund_num))

target-folder\English_EN\MaxiCharger AC\Compact\User Manual ... ['arch'] ... ['MaxiCharger AC Compact - User Manual_20230302_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Compact\Quick Guide ... ['arch'] ... ['MaxiCharger AC Compact - Quick Guide_20230215_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Ultra\User Manual ... ['arch'] ... ['MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt', 'MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf', 'Some - Random File_20220303_EN.pdf', 'MaxiCharger AC Ultra - User Manual EU_EN.pdf', 'MaxiCharger AC Ultra - User Manual_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Ultra\Quick Guide ... ['arch'] ... ['MaxiCharger AC Ultra - Quick Guide_20230330_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Wallbox\Quick Guide ... ['arch'] ... ['MaxiCharger AC Wallbox - Quick Guide_20230523_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Wallbox\Pedestal EP\Quick Guide ... ['arch'] ... ['MaxiCharger AC Wallbox Pedestal EP - Quick Guide_2023051

#### Function 1: path_matcher()
The function *path_matcher()* analizes the path as argument, extract the information that the name of document under this path **MUST** contain.

In [29]:
import re

#root = 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Pedestal EP\\Quick Guide'
#root = 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Installation and Operation Manual'
#root = 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Pedestal EP\\'
root = 'target-folder\\English_EN\\Energy Management System\\Operation and Installation Manual'
#root = 'target-folder\\English_EN\\MaxiCharger AC\\Maintenance Manual'

def path_extractor(root:str) -> tuple[str, str, str]:
    pattern_1 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))(.*)(\\.+ (Guide|Manual))$"
    pattern_2 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(Energy Management System)(.*)(\\.+ (Guide|Manual))$"
    match_1 = re.match(pattern_1, root)
    match_2 = re.match(pattern_2, root)
    if match_1:
        category = match_1.group(3)
        object = match_1.group(5).replace("\\", " ").strip()
        document = match_1.group(6).replace("\\", "")     
    elif match_2:
        category = match_2.group(3)
        object = ''
        document = match_2.group(5).replace("\\", "")  
    else:
        print("Path '%s' is not correct." % (root))
        category = ''
        object = ''
        document = ''
    return category, object, document

t = path_extractor(root)
print(t)

('Energy Management System', '', 'Operation and Installation Manual')


#### Function 2: file_matcher()
The function *file_matcher* analizes the *list of files* as argument under given path. Collect name and release date of pdf-file in a *dictionary*, who's name complies the naming regulation. The pdf-file that edited recently  

In [21]:
import os
import re

from datetime import datetime


files = ['MaxiCharger AC Ultra - User Manual EU_EN.pdf', 
         'MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt',
         'MaxiCharger AC Ultra - User Manual_EN.pdf', 
         'MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf',
         'Some - Random File_20220303_EN.pdf']
root = 'target-folder\\English_EN\\MaxiCharger AC\\Ultra\\User Manual'
path_info = ('MaxiCharger AC', 'Ultra', 'User Manual')


def file_matcher (root:str, path_info:tuple[str, str, str], files:list) -> tuple[str, str]:
    # initial dict of all existing pdf files
    pdf_files = {}
    result = False
    # get formal file name
    p1, p2, p3 = path_info
    file_name = (p1 + ' ' + p2 if len(p2)>0 else p1) + ' - ' + p3
    '''
    pattern_part = r"%s %s - %s(?:.*)?.*.pdf$" % path_info
    pattern_full = r"%s %s - %s(?:.*)?_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf$" % path_info
    pattern_pdf = r".*.pdf$"
    '''
    pattern_part = file_name + '(?:.*)?.*.pdf$'
    pattern_full = file_name + '(?:.*)?_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf$'
    # if appropriate pdf file exists under the 
    for file in files:
        # fuzzy match
        match_part = re.match(pattern_part, file, re.IGNORECASE)
        if match_part:
            result = True
            match_full = re.match(pattern_full, file, re.IGNORECASE)
            if match_full:
                release = f"{match_full.group(1)}-{match_full.group(2)}-{match_full.group(3)}"
            else:
                release = datetime.fromtimestamp(os.path.getmtime(os.path.join(root, file))).strftime(r"%Y-%m-%d")
            pdf_files.setdefault(file, release)
        else:
            #match_pdf = re.match(pattern_pdf, file)
            if file.endswith('.pdf'):
                print("Unregular named file: %s detected." % (file))
            else:
                print("Non-pdf file: %s detected." % (file))
            pass
    try:
        # returns full file name and its release date
        latest_file, latest_release = max(pdf_files.items(), key=lambda item: item[1])
        print(f"Latest file {latest_file} edited on {latest_release} verified.")
    except ValueError:
        latest_file = ''
        latest_release = ''
        print(f"Document {file_name} is missing.")
    return latest_file, latest_release, result

# 如果files为空，则返回null
m = file_matcher (root, path_info, files)
lf, lr, r = m
print(m)
print(lf, lr, r)

Non-pdf file: MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt detected.
Unregular named file: MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf detected.
Unregular named file: Some - Random File_20220303_EN.pdf detected.
Latest file MaxiCharger AC Ultra - User Manual_EN.pdf edited on 2023-07-06 verified.
('MaxiCharger AC Ultra - User Manual_EN.pdf', '2023-07-06', True)
MaxiCharger AC Ultra - User Manual_EN.pdf 2023-07-06 True


In [5]:
d = {'MaxiCharger AC Ultra - User Manual EU_EN.pdf': '2023-06-23', 'MaxiCharger AC Ultra - User Manual_20230706_EN.pdf': '2023-07-06'}
max_key, max_value = max(d.items(), key=lambda item: item[1])
print(max_key, max_value)

MaxiCharger AC Ultra - User Manual_20230706_EN.pdf 2023-07-06


In [6]:
from datetime import datetime

current_date = datetime.now().strftime(r"%Y-%m-%d")
print(current_date)

2023-07-29


#### Function 3: file_detector()
Combination of function 1 and function 2.

In [43]:
import os
import re

from datetime import datetime


####################################################################################################################
# Test Cases
test_cases = {
    'case 0': {
        'files': [],
        'root': 'target-folder\\English_EN\\MaxiCharger DC\\Fast\\CMS\\Quick Guide',
        'result': ('MaxiCharger DC', 'Fast CMS', 'Quick Guide', '', False)
    },
    'case_1': {
        'files': ['MaxiCharger AC Ultra - User Manual EU_EN.pdf', 
                  'MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt',
                  'MaxiCharger AC Ultra - User Manual_EN.pdf', 
                  'MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf', 
                  'Some - Random File_20220303_EN.pdf'],
        'root': 'target-folder\\English_EN\\MaxiCharger AC\\Ultra\\User Manual',
        'result': ('MaxiCharger AC', 'Ultra', 'User Manual', '2023-07-06', True)
    },
    'case_2': {
        'files': ['Energy Management System - Operation and Installation Manual EU_20230525_EN.pdf'],
        'root': 'target-folder\\English_EN\\Energy Management System\\Operation and Installation Manual',
        'result': ('Energy Management System', '', 'Operation and Installation Manual', '2023-05-25', True)
    },
    'case_3': {
        'files': ['MaxiCharger AC - Maintenance Manual EU_20220224_EN.pdf'],
        'root': 'target-folder\\English_EN\\MaxiCharger AC\\Maintenance Manual',
        'result': ('MaxiCharger AC', '', 'Maintenance Manual', '2022-02-24', True)
    },
    'case_4': {
        'files': ['MaxiCharger AC Wallbox Pedestal EP - Quick Guide_20230512_EN.pdf'],
        'root': 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Pedestal EP\\Quick Guide',
        'result': ('MaxiCharger AC', 'Wallbox Pedestal EP', 'Quick Guide', '2023-05-12', True)
    },
}
####################################################################################################################


def file_detector (root:str, files:list) -> tuple[str, str, str, str, bool]:
    ##########################################################
    # Part 1: Path Extraction
    ##########################################################
    pattern_1 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))(.*)(\\.+ (Guide|Manual))$"
    pattern_2 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(Energy Management System)(.*)(\\.+ (Guide|Manual))$"
    match_1 = re.match(pattern_1, root)
    match_2 = re.match(pattern_2, root)
    try:
        if match_1:
            category = match_1.group(3)
            item = match_1.group(5).replace("\\", " ").strip()
            document = match_1.group(6).replace("\\", "")     
        elif match_2:
            category = match_2.group(3)
            item = ''
            document = match_2.group(5).replace("\\", "")  
    except AttributeError:
        print("Path '%s' is not a proper document path." % (root))
        category = 'null'
        item = 'null'
        document = 'null'
    ##########################################################
    # Part 2: File Matching
    ##########################################################
    pdf_files = {}
    fund = False
    # get formal file name
    formal_name = (category + ' ' + item if len(item)>0 else category) + ' - ' + document
    '''
    pattern_part = r"%s %s - %s(?:.*)?.*.pdf$" % path_info
    pattern_full = r"%s %s - %s(?:.*)?_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf$" % path_info
    pattern_pdf = r".*.pdf$"
    '''
    pattern_part = formal_name + '(?:.*)?.*.pdf$'
    pattern_full = formal_name + '(?:.*)?_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf$'
    # if appropriate pdf file exists under the 
    for file in files:
        # fuzzy match
        match_part = re.match(pattern_part, file, re.IGNORECASE)
        if match_part:
            fund = True
            # accurate match
            match_full = re.match(pattern_full, file, re.IGNORECASE)
            if match_full:
                release = f"{match_full.group(1)}-{match_full.group(2)}-{match_full.group(3)}"
            else:
                release = datetime.fromtimestamp(os.path.getmtime(os.path.join(root, file))).strftime(r"%Y-%m-%d")
            pdf_files.setdefault(file, release)
        else:
            #match_pdf = re.match(pattern_pdf, file)
            if file.endswith('.pdf'):
                print("Unregular named file: %s detected." % (file))
            else:
                print("Non-pdf file: %s detected." % (file))
            pass
    # returns detection result
    try:
        _, latest_release = max(pdf_files.items(), key=lambda item: item[1])
        print(f"Document {formal_name} edited on {latest_release} verified.")
    except ValueError:
        latest_release = ''
        print(f"Document {formal_name} is missing.")
    return category, item, document, latest_release, fund


####################################################################################################################
# Test
for case, content in test_cases.items():
    test_result = file_detector(content['root'], content['files'])
    if content['result'] == test_result:
        print(f"{case} passed.")
    else:
        print(f"{case} failed.")
        pass
####################################################################################################################

Document MaxiCharger DC Fast CMS - Quick Guide is missing.
case 0 passed.
Non-pdf file: MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt detected.
Unregular named file: MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf detected.
Unregular named file: Some - Random File_20220303_EN.pdf detected.
Document MaxiCharger AC Ultra - User Manual edited on 2023-07-06 verified.
case_1 passed.
Document Energy Management System - Operation and Installation Manual edited on 2023-05-25 verified.
case_2 passed.
Document MaxiCharger AC - Maintenance Manual edited on 2022-02-24 verified.
case_3 passed.
Document MaxiCharger AC Wallbox Pedestal EP - Quick Guide edited on 2023-05-12 verified.
case_4 passed.


#### Function 4: file_analyzer()
The function file extractor get the edit and release times, also the language code of the selected pdf file.

In [8]:
import os
import re
import csv

from datetime import datetime

# 目标文件夹根目录
target_folder = "target-folder"  

def file_recorder (folders: list):
    for folder in folder:
    match = re.match(r"([A-Z][a-zA-Z]+)_(\w{2})", folder)
    language = match.group(1)  # English
    language_code = match.group(2)  # EN

    csv_name = f"{langu_code}_log_{}.csv"  # EN_log.csv
    csv_path = os.path.join(target_folder, folder, csv_file)
    doc_num = 0
    fund_num = 0
    with open(csv_path, 'w', newline='') as csv_file:
    


SyntaxError: incomplete input (3599511859.py, line 6)

In [44]:
import os
import csv
import re

from datetime import datetime

# 目标文件夹根目录
target_folder = "target-folder"  

# 获取所有语言分类子文件夹的路径
#lang_folders = [folder for folder in os.listdir(target_folder) if os.path.isdir(os.path.join(target_folder, folder))]
#lang_folders.remove('English_EN')

lang_folder = 'English_EN'
lang_code = re.match(r"([A-Z][a-zA-Z]+)_(\w{2})", lang_folder).group(2)  # EN

csv_file = f"{lang_code}_log.csv"  # EN_log.csv
csv_path = os.path.join(target_folder, lang_folder, csv_file)

"""
"""
doc_num = 0
fund_num = 0
# 打开CSV文件并写入表头
with open(csv_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['language', 'category', 'object', 'document', 'release', 'status'])
    # 遍历目标文件夹下的所有内容
    for root, dirs, files in os.walk(os.path.join(target_folder, lang_folder)):
        # 只处理包含arch子项的文件夹
        if "arch" in dirs:
            doc_num += 1
            try:
                category, item, document, latest_release, fund = file_detector (root, files)
                if fund:
                    status = "verified"
                    fund_num += int(fund)
                else:
                    status = "missing"
            except NameError as error:
                print(f"An error occurred: {error}")
                pass
        else:
            continue
        writer.writerow([lang_code, category, item, document, latest_release, status])

print("%d documents shall be available." % (doc_num))
print("%d documents has been found." % (fund_num))
        


Document MaxiCharger AC Compact - User Manual edited on 2023-03-02 verified.
Document MaxiCharger AC Compact - Quick Guide edited on 2023-02-15 verified.
Non-pdf file: MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt detected.
Unregular named file: MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf detected.
Unregular named file: Some - Random File_20220303_EN.pdf detected.
Document MaxiCharger AC Ultra - User Manual edited on 2023-07-06 verified.
Document MaxiCharger AC Ultra - Quick Guide edited on 2023-03-30 verified.
Document MaxiCharger AC Wallbox - Quick Guide edited on 2023-05-23 verified.
Document MaxiCharger AC Wallbox Pedestal EP - Quick Guide edited on 2023-05-12 verified.
Document MaxiCharger AC Wallbox - Installation and Operation Manual edited on 2023-07-17 verified.
Document MaxiCharger AC Pedestal - Quick Guide edited on 2023-02-23 verified.
Document MaxiCharger AC - Maintenance Manual edited on 2022-02-24 verified.
Document MaxiCharger DC Compact - Installation

#### Function 4: md_writer()
The function file extractor get the edit and release times, also the language code of the selected pdf file.

In [132]:
import os
import csv

from datetime import datetime

target_folder = "target-folder" 
lang_folders = ['English_EN']

# 生成 README.md 文件
readme_path = os.path.join(target_folder, "README.md")

with open(readme_path, "w") as readme_file:
    readme_file.write("# verification result\n\n")
    readme_file.write("# verification result\n\n")
    for lang_folder in lang_folders:
        lang_code = lang_folder.split("_")[-1]
        csv_file = f"{lang_code}_verification.csv"
        csv_path = os.path.join(lang_folder, csv_file)

        readme_file.write(f"## {lang_code}\n\n")
        readme_file.write(f"| language | category | object | document | update | status |\n")
        readme_file.write(f"| --- | --- | --- | --- | --- | --- |\n")
        readme_file.write(f"| [CSV file](./{csv_path}) |\n\n")

['f1.pdf']


In [13]:
import os

test_folder = "test-folder" 
rm_path = os.path.join(test_folder, "README.md")

values = ("v1", "v2", "v3", "v4", "v5", "v6")

with open(rm_path, "w") as rm_file:
    rm_file.write("# Titel level 1\n")
    rm_file.write("## Titel level 2\n")
    rm_file.write("### Titel level 3\n")
    rm_file.write("#### Titel level 4\n")
    rm_file.write("first line<br>\n")
    rm_file.write("second line<br>\n\n")
    rm_file.write(f"| language | category | object | document | update | status |\n")
    rm_file.write(f"| --- | --- | --- | --- | --- | --- |\n")
    for i in range(10):
        rm_file.write(f"| {values[0]} | {values[1]} | {values[2]} | {values[3]} | {values[4]} | {values[5]} |\n")
    

In [ ]:
import os

# 原始目录和目标目录
source_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"
target_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"

# 遍历源目录下的所有文件
for filename in os.listdir(source_dir):
    if filename.endswith(".pdf"):
        # 构建源文件路径和目标文件路径
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)
        
        # 将目标文件路径中的文件名替换为指定的 filename
        target_file = os.path.join(target_dir, filename.replace(".pdf", "_" + filename))
        
        # 重命名文件
        os.rename(source_file, target_file)